In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import transformers
import load_data
import torch
from torch import nn
from transformers import BertModel, BertTokenizer, CamembertModel, CamembertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import numpy as np
import os
from training_audio_model import *
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') # GPU ou CPU
print(torch.cuda.is_available())
# Chargement des données
transcr_path='paco-cheese/transcr'
data=load_data.load_all_ipus(folder_path=transcr_path,load_words=True)

data[:3]

C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


True


,ipu_id,speaker,start_ipu,stop_ipu,text_ipu,is_main_speaker,turn_at_start,turn_after,turn_start_word,yield_at_end,request_at_start,start_words,stop_words,duration,text_words,request_after_word,turn_after_word,is_ipu_end,dyad
0,0,AA,4.54,4.840,tu as,True,False,True,4.84,True,False,4.54,4.72,0.18,tu,False,False,False,transcr\AAOR
1,0,AA,4.54,4.840,tu as,True,False,True,4.84,True,False,4.72,4.84,0.12,as,False,False,True,transcr\AAOR
2,1,OR,5.14,5.825,mh ouais si tu veux,True,True,False,NaN,False,True,5.14,5.44,0.30,mh,True,False,False,transcr\AAOR


In [2]:
# Load the data into a DataFrame
df = pd.DataFrame(data)
from utils import *
y=create_y(df) # Création de la target


# Importation des données audios

In [3]:

audio_files_path = 'paco-cheese/audio/2_channels/'
audio_segments = extract_audio_segments(data,audio_files_path)
X = np.array([extract_features(segment) for segment in audio_segments])


paco-cheese/audio/2_channels/AA_OR.wav
982587
paco-cheese/audio/2_channels/AC_MZ.wav
897019
paco-cheese/audio/2_channels/AS_EP.wav
965947
paco-cheese/audio/2_channels/AW_CG.wav
1033723
paco-cheese/audio/2_channels/BE_CR.wav
1036379
paco-cheese/audio/2_channels/CM_MCC.wav
969659
paco-cheese/audio/2_channels/ER_AG.wav
892283
paco-cheese/audio/2_channels/FB_CB.wav
1119451
paco-cheese/audio/2_channels/FS_MG.wav
932987
paco-cheese/audio/2_channels/JA_EA.wav
1012361
paco-cheese/audio/2_channels/JDS_LS.wav
1056539
paco-cheese/audio/2_channels/LJ_JL.wav
1591419
paco-cheese/audio/2_channels/JR_BG.wav
1109339
paco-cheese/audio/2_channels/JS_CL.wav
991483
paco-cheese/audio/2_channels/LB_MA.wav
1055995
paco-cheese/audio/2_channels/LE_LB.wav
1244379
paco-cheese/audio/2_channels/LJ_JL.wav
1591419
paco-cheese/audio/2_channels/LP_MA.wav
1183131
paco-cheese/audio/2_channels/LS_NA.wav
1302523
paco-cheese/audio/2_channels/MA_PC.wav
1044155
paco-cheese/audio/2_channels/MC_MRH.wav
987259
paco-cheese/audio/

C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1600
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1280
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1920
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=13

In [16]:
dataset = AudioDataset(X, y)
train_size=int(0.8*len(dataset))
X_train = dataset[:train_size]
y_train = y[:train_size]


X_test = dataset[train_size:]
y_test = y[train_size:]

# Divisez les données en ensembles d'entraînement et de test
#train_size = int(0.8 * len(dataset))
#test_size = len(dataset) - train_size
#train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

train_dataset = AudioDataset(X_train, y_train)
test_dataset = AudioDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader_audio = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [5]:
print(len(y))
print(len(y_test))

110544
22109


In [7]:
model=torch.load('modele/model_audio')
all_preds_audio,all_labels=predition_model_audio(model,test_loader_audio,device,proba=False)
f1 = f1_score(all_labels, all_preds_audio, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_preds_audio)
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])
detected_class_0 = conf_matrix[0, 0]  # Vrais positifs pour la classe 0
detected_class_1 = conf_matrix[1, 1]  # Vrais positifs pour la classe 1

print(f'Nombre d\'éléments de classe 0 détectés : {detected_class_0} sur {total_class_0}')
print(f'Nombre d\'éléments de classe 1 détectés : {detected_class_1} sur {total_class_1}')


Test F1 Score: 0.8584121695443728
Confusion Matrix:
[[17554  2776]
 [  900   879]]
Nombre d'éléments de classe 0 détectés : 17554 sur 20330
Nombre d'éléments de classe 1 détectés : 879 sur 1779


In [8]:
#model = importation

all_preds_audio,all_labels_audio=predition_model_audio(model,test_loader_audio,device,proba=True)
print(all_preds_audio)
print(len(all_preds_audio))

[[ 0.06999674  0.10308494]
 [ 0.5474191  -0.5814937 ]
 [ 0.57590795 -0.61488396]
 ...
 [ 0.61825335 -0.96196795]
 [ 0.7414569  -1.1085435 ]
 [-1.0408167   1.3075197 ]]
22109


# Importation données texte

In [17]:
from training_text_model import *


features = data['text_words'].to_numpy().reshape(-1).tolist()  

features = [word if not pd.isnull(word) else '[UNK]' for word in features]
train_size = int(len(features) * 0.8)
train_size=train_size
X_train = features[:train_size]
y_train = y[:train_size]

indices = [i for i, x in enumerate(y_train) if x == 1]

X_test = features[train_size:]
y_test = y[train_size:]

window_size = 5


X_train = create_sequences(X_train)
X_test = create_sequences(X_test)
# Paramètres
model_name = 'camembert-base'
max_length = 256
batch_size = 16
epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Tokenizer pour CamemBERT
tokenizer = CamembertTokenizer.from_pretrained(model_name)


# Création des datasets et dataloaders pour le texte
train_dataset = TextDataset(X_train, y_train, tokenizer, max_length)
train_loader_text = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TextDataset(X_test, y_test, tokenizer, max_length)
test_loader_text = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## prediction modèle texte

In [11]:
all_preds_text = []
all_labels = []
model= torch.load('modele/camembert_epoch_3.bin')
model.eval()
all_preds_text,all_labels = prediction_model_text(model,test_loader_text,device,proba=False)
f1 = f1_score(all_labels, all_preds_text, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_preds_text)
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])
detected_class_0 = conf_matrix[0, 0]  # Vrais positifs pour la classe 0
detected_class_1 = conf_matrix[1, 1]  # Vrais positifs pour la classe 1

print(f'Nombre d\'éléments de classe 0 détectés : {detected_class_0} sur {total_class_0}')
print(f'Nombre d\'éléments de classe 1 détectés : {detected_class_1} sur {total_class_1}')

Test F1 Score: 0.7958142033852643
Confusion Matrix:
[[15591  4739]
 [  950   829]]
Nombre d'éléments de classe 0 détectés : 15591 sur 20330
Nombre d'éléments de classe 1 détectés : 829 sur 1779


# prédiction early fusion

In [18]:

model_text= torch.load('modele/camembert_epoch_3.bin')
model_text.eval()
all_preds_text_proba,all_labels = prediction_model_text(model_text,test_loader_text,device,proba=True)

#combined_preds_and = [pred_text and pred_audio for pred_text, pred_audio in zip(all_preds_text, all_preds_audio)]


KeyboardInterrupt: 

In [32]:
model_audio=torch.load('modele/model_audio')
import torch
import numpy as np


all_preds_audio_proba,all_labels_audio=predition_model_audio(model_audio,test_loader_audio,device,proba=True)

tensor([[ 0.0700,  0.1031],
        [ 0.5474, -0.5815],
        [ 0.5759, -0.6149],
        [ 0.5824, -0.6302],
        [ 0.6023, -0.6569],
        [ 0.6102, -0.6524],
        [ 0.4579, -0.3790],
        [ 0.3298,  0.0492],
        [ 0.1920,  0.1933],
        [ 0.3428, -0.3276],
        [ 0.4739, -0.5035],
        [ 1.5697, -1.9588],
        [-0.1893,  1.0620],
        [ 0.0696,  0.4555],
        [ 1.1094, -1.4019],
        [ 1.2694, -1.5883],
        [ 1.4976, -1.9248],
        [ 0.8441, -0.8726],
        [ 0.6722, -0.6439],
        [ 1.2707, -1.5216],
        [ 1.2939, -1.5639],
        [ 0.1557,  0.3026],
        [ 0.8280, -0.8496],
        [ 0.1317,  0.1879],
        [ 0.1699,  0.0366],
        [ 0.6429, -0.5348],
        [ 1.0417, -1.0249],
        [ 0.2903, -0.1148],
        [ 1.7763, -1.9930],
        [ 0.7022, -0.7424],
        [ 0.7670, -0.8817],
        [ 1.0312, -1.1755]], device='cuda:0')
tensor([[ 0.2537,  0.0772],
        [ 0.1501,  0.0831],
        [ 0.1589,  0.0886],
  

In [33]:
print(all_preds_audio)
print(all_preds_text)


[[ 0.06999674  0.10308494]
 [ 0.5474191  -0.5814937 ]
 [ 0.57590795 -0.61488396]
 ...
 [ 0.61825335 -0.96196795]
 [ 0.7414569  -1.1085435 ]
 [-1.0408167   1.3075197 ]]
[[ 0.38150686 -0.4166726 ]
 [ 0.64231646 -0.6344916 ]
 [ 0.6327969  -0.62154025]
 ...
 [ 0.3585049  -0.3926447 ]
 [-0.7127487   0.62089145]
 [-0.71275014  0.6208931 ]]


In [43]:
import numpy as np
# Addition des tableaux de probabilités
sum_array = np.add(all_preds_audio_proba, all_preds_text_proba)

# Trouver la classe pour chaque liste (0 ou 1)
combined_preds = np.argmax(sum_array, axis=1)


In [44]:
f1 = f1_score(all_labels, combined_preds, average='weighted')
conf_matrix = confusion_matrix(all_labels, combined_preds)
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')


total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])
detected_class_0 = conf_matrix[0, 0]  # Vrais positifs pour la classe 0
detected_class_1 = conf_matrix[1, 1]  # Vrais positifs pour la classe 1

print(f'Nombre d\'éléments de classe 0 détectés : {detected_class_0} sur {total_class_0}')
print(f'Nombre d\'éléments de classe 1 détectés : {detected_class_1} sur {total_class_1}')



Test F1 Score: 0.871181950459075
Confusion Matrix:
[[18068  2262]
 [  964   815]]
Nombre d'éléments de classe 0 détectés : 18068 sur 20330
Nombre d'éléments de classe 1 détectés : 815 sur 1779
